In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.models import mobilenet_v2

In [2]:
# 转换器
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# 添加标签噪声的函数
def add_label_noise(labels, noise_type='symmetric', noise_rate=0.2):
    noisy_labels = labels.copy()
    n_samples = labels.shape[0]
    n_classes = 10

    if noise_type == 'symmetric':
        n_noisy = int(noise_rate * n_samples)
        noise_idx = np.random.choice(n_samples, n_noisy, replace=False)
        for idx in noise_idx:
            original_label = labels[idx]
            noisy_labels[idx] = np.random.choice([l for l in range(n_classes) if l != original_label])
    elif noise_type == 'pairflip':
        for i in range(n_classes // 2):
            flip_from = i * 2
            flip_to = flip_from + 1
            flip_idx = np.where(labels == flip_from)[0]
            n_flip = int(noise_rate * len(flip_idx))
            flip_idx = np.random.choice(flip_idx, n_flip, replace=False)
            noisy_labels[flip_idx] = flip_to

    return noisy_labels

In [5]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [10]:
# MobileNet 模型
net = mobilenet_v2(pretrained=False)
net.classifier[1] = nn.Linear(net.classifier[1].in_features, 10)
net.to(device)
print("start")

# 训练模型
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # 遍历数据集多次
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #labels = labels.numpy()
        labels = labels.cpu().numpy()
        # 添加噪声
        noisy_labels = add_label_noise(labels, noise_type='symmetric', noise_rate=0.2)
        noisy_labels = torch.from_numpy(noisy_labels).to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, noisy_labels.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
         # 打印当前进度
        if i % 200 == 199:    # 每 200 批次打印一次
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}/{len(trainloader)}, Loss: {running_loss / 200}')
            running_loss = 0.0
            

    print(f'Epoch {epoch + 1} loss: {running_loss / len(trainloader)}')

print('Finished Training')

# 评估模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

start
Epoch: 1, Batch: 200/12500, Loss: 2.5420664983987806
Epoch: 1, Batch: 400/12500, Loss: 2.6281258481740952
Epoch: 1, Batch: 600/12500, Loss: 2.70054610311985
Epoch: 1, Batch: 800/12500, Loss: 2.626032825112343
Epoch: 1, Batch: 1000/12500, Loss: 2.6366637283563614
Epoch: 1, Batch: 1200/12500, Loss: 2.6146209037303922
Epoch: 1, Batch: 1400/12500, Loss: 2.7034409976005556
Epoch: 1, Batch: 1600/12500, Loss: 2.6924518477916717
Epoch: 1, Batch: 1800/12500, Loss: 2.6725606322288513
Epoch: 1, Batch: 2000/12500, Loss: 2.671718465089798
Epoch: 1, Batch: 2200/12500, Loss: 2.661849749684334
Epoch: 1, Batch: 2400/12500, Loss: 2.648248463869095
Epoch: 1, Batch: 2600/12500, Loss: 2.640688962340355
Epoch: 1, Batch: 2800/12500, Loss: 2.6533773535490037
Epoch: 1, Batch: 3000/12500, Loss: 2.589056440591812
Epoch: 1, Batch: 3200/12500, Loss: 2.6714760208129884
Epoch: 1, Batch: 3400/12500, Loss: 2.5877103769779204
Epoch: 1, Batch: 3600/12500, Loss: 2.6774789929389953
Epoch: 1, Batch: 3800/12500, Loss: